In [2]:
#wandb
from kaggle_secrets import UserSecretsClient
import wandb
user_secrets = UserSecretsClient()

my_secret = user_secrets.get_secret("bert-lstm") 

wandb.login(key=my_secret)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: storknguyen-286 (storknguyen-286-university-system-of-georgia) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [3]:
!gdown 18ZT8T7cUHvHNqoe_aZiDslR07lodEOK_
!unzip sentiment.zip
!rm sentiment.zip 
!ls

Downloading...
From (original): https://drive.google.com/uc?id=18ZT8T7cUHvHNqoe_aZiDslR07lodEOK_
From (redirected): https://drive.google.com/uc?id=18ZT8T7cUHvHNqoe_aZiDslR07lodEOK_&confirm=t&uuid=2d5a06d6-413b-4f54-9592-dec8fccb5566
To: /kaggle/working/sentiment.zip
100%|████████████████████████████████████████| 672M/672M [00:22<00:00, 30.1MB/s]
Archive:  sentiment.zip
  inflating: multi_cased_L-12_H-768_A-12/vocab.txt  
  inflating: multi_cased_L-12_H-768_A-12/bert_config.json  
  inflating: multi_cased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: multi_cased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: NTC_SV/NTC_SV_test.csv  
  inflating: NTC_SV/NTC_SV_train.csv  
  inflating: multi_cased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
multi_cased_L-12_H-768_A-12  NTC_SV


In [4]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
from tqdm import tqdm
import os
from safetensors.torch import save_file

2025-04-26 12:54:53.202056: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745672093.361802      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745672093.411709      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

Using device: cuda


In [6]:
SEQ_LEN = 256
BATCH_SIZE = 16
EPOCHS = 10
LR = 2e-5
SAVE_EVERY = 2

In [7]:
df = pd.read_csv("/kaggle/working/NTC_SV/NTC_SV_train.csv").dropna()
texts = df['review'].tolist()
labels = LabelEncoder().fit_transform(df['label'])

In [8]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [9]:
class FoodyDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx],
            truncation=True,
            padding='max_length',
            max_length=self.max_len,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'label': torch.tensor(self.labels[idx], dtype=torch.long)
        }

In [10]:
X_train, X_val, y_train, y_val = train_test_split(texts, labels, test_size=0.15, random_state=42)
train_dataset = FoodyDataset(X_train, y_train, tokenizer, SEQ_LEN)
val_dataset = FoodyDataset(X_val, y_val, tokenizer, SEQ_LEN)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

In [11]:
df_test = pd.read_csv("/kaggle/working/NTC_SV/NTC_SV_test.csv").dropna()
X_test = df_test['review'].tolist()
y_test = LabelEncoder().fit(df_test['label']).transform(df_test['label'])
test_dataset = FoodyDataset(X_test, y_test, tokenizer, SEQ_LEN)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [12]:
class BERTLSTMClassifier(nn.Module):
    def __init__(self, hidden_dim=128, num_classes=3):
        super().__init__()
        self.bert = BertModel.from_pretrained('bert-base-multilingual-cased')
        self.lstm = nn.LSTM(768, hidden_dim, batch_first=True, bidirectional=True)
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(hidden_dim * 2, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        lstm_out, _ = self.lstm(outputs.last_hidden_state)
        pooled = torch.mean(lstm_out, dim=1)
        out = self.dropout(pooled)
        return self.fc(out)

In [13]:
!pip install huggingface_hub[hf_xet]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 MB 33.3 MB/s eta 0:00:00:00:0100:01


In [14]:
model = BERTLSTMClassifier(num_classes=len(set(labels))).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
criterion = nn.CrossEntropyLoss()

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

In [15]:
import os
import torch
from tqdm import tqdm
from sklearn.metrics import accuracy_score
import wandb
from safetensors.torch import save_file  # nếu dùng safetensors

def train_model(
    model,
    data_loader,
    optimizer,
    criterion,
    device,
    epochs: int,
    save_every: int,
    save_dir: str,
    project_name: str,
    model_name: str
):
    # Chuẩn bị thư mục lưu checkpoint
    os.makedirs(save_dir, exist_ok=True)

    # Khởi tạo W&B
    wandb.init(
        project=project_name,
        name=model_name,
        config={
            "epochs": epochs,
            "batch_size": data_loader.batch_size,
            "learning_rate": optimizer.param_groups[0]['lr'],
            "model": model_name,
        }
    )

    for epoch in range(epochs):
        model.train()
        total_loss = 0.0
        all_preds = []
        all_labels = []

        loop = tqdm(data_loader, desc=f"{model_name} Epoch {epoch+1}/{epochs}")
        for batch in loop:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # Cộng dồn loss & metrics
            total_loss += loss.item()
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            # Log batch loss ngay lập tức
            wandb.log({f"{model_name}_batch_loss": loss.item()})

        # Tính toán loss/acc của epoch
        epoch_loss = total_loss / len(data_loader)
        epoch_acc = accuracy_score(all_labels, all_labels) if len(all_labels)>0 else 0.0  # lưu ý sửa thành all_preds
        epoch_acc = accuracy_score(all_labels, all_preds)

        # Log epoch metrics
        wandb.log({
            f"{model_name}_epoch_loss": epoch_loss,
            f"{model_name}_epoch_accuracy": epoch_acc,
            "epoch": epoch + 1
        })

        print(f"\n✅ {model_name} Epoch {epoch+1}/{epochs} — Loss: {epoch_loss:.4f} — Accuracy: {epoch_acc:.2%}")

        # Lưu checkpoint mỗi save_every epochs
        if (epoch + 1) % save_every == 0:
            ckpt_name = f"{model_name}_epoch{epoch+1}.safetensors"
            ckpt_path = os.path.join(save_dir, ckpt_name)
            save_file(model.state_dict(), ckpt_path)
            print(f"💾 Saved checkpoint at {ckpt_path}")
            wandb.save(ckpt_path)

    wandb.finish()


In [16]:
train_model(
    model=model,
    data_loader=train_loader,
    optimizer=optimizer,
    criterion=criterion,
    device=device,
    epochs=EPOCHS,
    save_every=SAVE_EVERY,
    save_dir="/kaggle/working",
    project_name="bert-lstm",
    model_name="bert-lstm"
)


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


bert-lstm Epoch 1/10: 100%|██████████| 2166/2166 [16:49<00:00,  2.15it/s]



✅ bert-lstm Epoch 1/10 — Loss: 0.3166 — Accuracy: 86.93%


bert-lstm Epoch 2/10: 100%|██████████| 2166/2166 [16:48<00:00,  2.15it/s]



✅ bert-lstm Epoch 2/10 — Loss: 0.2421 — Accuracy: 90.50%


...

bert-lstm Epoch 9/10: 100%|██████████| 2166/2166 [16:45<00:00,  2.15it/s]



✅ bert-lstm Epoch 9/10 — Loss: 0.0686 — Accuracy: 97.77%


bert-lstm Epoch 10/10: 100%|██████████| 2166/2166 [16:45<00:00,  2.15it/s]



✅ bert-lstm Epoch 10/10 — Loss: 0.0551 — Accuracy: 98.19%
💾 Saved checkpoint at /kaggle/working/bert-lstm_epoch10.safetensors


bert-lstm_batch_loss,▇█▅▂▄▃▅▄▂▁▂▅▄▅▃▁▁▃▆▃▁▁▁▂▁▄▂▁▁▁▁▁▁▂▁▁▁▁▂▂
bert-lstm_epoch_accuracy,▁▃▄▅▆▇▇▇██
bert-lstm_epoch_loss,█▆▅▄▃▃▂▂▁▁
epoch,▁▂▃▃▄▅▆▆▇█
bert-lstm_batch_loss,0.12224
bert-lstm_epoch_accuracy,0.98187
bert-lstm_epoch_loss,0.05508
epoch,10


In [17]:
def evaluate(model, data_loader):
    model.eval()
    predictions, true_labels = [], []
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].cuda()
            attention_mask = batch['attention_mask'].cuda()
            labels = batch['label'].cuda()

            outputs = model(input_ids, attention_mask)
            preds = torch.argmax(outputs, dim=1)

            predictions.extend(preds.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    print(classification_report(true_labels, predictions))

print("=== Evaluation on Test Set ===")
evaluate(model, test_loader)



=== Evaluation on Test Set ===
              precision    recall  f1-score   support

           0       0.88      0.90      0.89      5000
           1       0.90      0.88      0.89      5000

    accuracy                           0.89     10000
   macro avg       0.89      0.89      0.89     10000
weighted avg       0.89      0.89      0.89     10000



In [18]:
save_file(model.state_dict(), "bert_lstm_foody.safetensors")